# Skforecast 0.14 migration guide

This document contains a set of instructions on how to update your code to work with Skforecast 0.14.

## Changes in the import of the library

Skforecast modules have been reorganized and renamed to make it more intuitive. The new import of the library is as follows:

| **Before**                                        | **Now**                                      |
|--------------------------------------------------|---------------------------------------------|
| `from skforecast.ForecasterAutoreg import ForecasterAutoreg`          | `from skforecast.autoreg import ForecasterRecursive` |
| `from skforecast.ForecasterAutoregCustom import ForecasterAutoregCustom` | Not exist any more, use `ForecasterRecursive` with window features  |
| `from skforecast.ForecasterAutoregDirect import ForecasterAutoregDirect` | `from skforecast.direct import ForecasterDirect`     |


## Custom features

Forecasters `ForecasterAutoregCustom` and `ForecasterAutoregDirectCustom` have been removed. Now, you can use `ForecasterRecursive` and `ForecasterDirect` with window features.

## Changes in the backtesting

The arguments `initial_train_size`, `window_size`, `differentiation`, `refit`, `fixed_train_size`, `gap`, `skip_folds`, `allow_incomplete_fold`, `return_all_indexes`, and `verbose` are no longer defined in the `backtesting_forecaster` method. Instead, an instance of `TimeSeriesFolds` should be created, and these arguments should be specified in the class constructor. This change not only allows the same folds to be reused across different methods, but also provides the ability to extract fold indexes.


| **Column 1** | **Column 2** |
|--------------|--------------|
| <pre>from skforecast.model_selection import backtesting_forecaster<br>backtesting_forecaster(<br>    forecaster            = forecaster,<br>    y                     = y,<br>    steps                 = 10,<br>    initial_train_size    = 100,<br>    metric                = 'mean_absolute_error',<br>    fixed_train_size      = True,<br>    gap                   = 0,<br>    skip_folds            = None,<br>    allow_incomplete_fold = True,<br>    refit                 = False,<br>    n_jobs                = 'auto',<br>    verbose               = False,<br>    show_progress         = True<br>)</pre> | <pre>from skforecast.model_selection import backtesting_forecaster<br>from skforecast.model_selection import TimeSeriesFold<br><br>cv = TimeSeriesFold(<br>    steps                 = 10,<br>    initial_train_size    = 100,<br>    fixed_train_size      = True,<br>    gap                   = 0,<br>    skip_folds            = None,<br>    allow_incomplete_fold = True,<br>    refit                 = False,<br>)<br><br>backtesting_forecaster(<br>    forecaster            = forecaster,<br>    y                     = y,<br>    cv                    = cv,<br>    metric                = 'mean_absolute_error',<br>    n_jobs                = 'auto',<br>    verbose               = False,<br>    show_progress         = True<br>)</pre> |


In [ ]:
# Skforecast <= 0.13
# ==============================================================================
from skforecast.model_selection import backtesting_forecaster

backtesting_forecaster(
    forecaster            = forecaster,
    y                     = y,
    steps                 = 10,
    initial_train_size    = 100,
    metric                = 'mean_absolute_error',
    fixed_train_size      = True,
    gap                   = 0,
    skip_folds            = None,
    allow_incomplete_fold = True,
    refit                 = False,
    n_jobs                = 'auto',
    verbose               = False,
    show_progress         = True
)

In [ ]:
# Skforecast >= 0.14
# ==============================================================================
from skforecast.model_selection import (
    backtesting_forecaster,
    TimeSeriesFold
)

cv = TimeSeriesFold(
    steps                 = 10,
    initial_train_size    = 100,
    fixed_train_size      = True,
    gap                   = 0,
    skip_folds            = None,
    allow_incomplete_fold = True,
    refit                 = False,
)

backtesting_forecaster(
    forecaster            = forecaster,
    y                     = y,
    cv                    = cv,
    metric                = 'mean_absolute_error',
    n_jobs                = 'auto',
    verbose               = False,
    show_progress         = True
)